2 main distributions:

number of voters per cluster: e.g. (1000,1000,750,500,200,200,100,100,100)

number of votes per project: e.g. (7000,6500,6000,4500,4000,3000,2000,2000,1500)

2 sub distributions:

number of projects in a cluster vs cluster size

number of approvals per voter: based on first two distributions



In [8]:
import numpy as np
from numpy import random
import copy 
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import umap
from collections import Counter

import statistics
from pylab import rcParams
from random import sample

In [9]:
class data_generator():
    def __init__(self, 
                 voters_per_cluster, 
                 votes_per_project, 
                 cluster_independence=3, 
                 spread_of_approvals=1.5, 
                 adcavpd=0.0, 
                 sdcavpd=0.1,
                 noise=0.01):
        self.voters_per_cluster = np.array(sorted(voters_per_cluster, reverse=True))
        self.votes_per_project = np.array(votes_per_project)
        self.cluster_independence = cluster_independence
        self.spread_of_approvals = spread_of_approvals
        self.avg_diff_cluster_all_voters_project_dist = adcavpd
        self.std_diff_cluster_all_voters_project_dist = sdcavpd
        self.noise = noise

        self.n_projects = len(votes_per_project)
        self.max_voters_per_cluster = max(voters_per_cluster)
        self.n_clusters = len(self.voters_per_cluster)
        self.rng = random.default_rng()
    
    @property
    def size_none_empty_projects(self):
        return len(self.none_empty_projects)
    
    @property
    def none_empty_projects(self):
        return np.where(self.votes_per_project != 0)[0]

    @property
    def votes_per_project_dist(self):
        return self.votes_per_project / sum(self.votes_per_project)
        
    @property
    def avg_approvals_per_voter(self):
        return sum(self.votes_per_project) / sum(self.voters_per_cluster)

    def convert_ballot_to_one_hot(self, ballots):
        new_ballots = np.zeros((len(ballots), self.n_projects))
        for i, vote in enumerate(ballots):
            new_ballots[i,vote] = 1
        return new_ballots
    
    def make_clusters(self):
        clusters = []
        for cluster_size in self.voters_per_cluster:
#             print(self.avg_approvals_per_voter)
#             print(self.votes_per_project)

#             print(self.votes_per_project_dist)
            npic = self.n_projects_in_cluster(cluster_size)
#             print(npic)
            ps = self.generate_project_set(npic)
#             print(ps)
            apv = self.approvals_per_voter(npic, cluster_size)
#             print(apv)
            cpd = self.gen_cluster_project_dist(ps)
#             print(cpd)
            cpdn,psn = self.gen_cluster_project_dist_noise(cpd, ps)
#             print(cpdn)
            ballots = self.gen_cluster_ballot(psn, apv, cpdn)
               
            # update generator
            for proj, votes in Counter([y for x in ballots for y in x]).items():
                self.votes_per_project[proj] = np.maximum(self.votes_per_project[proj] - votes, 0)
            self.voters_per_cluster = np.delete(self.voters_per_cluster, 0)
                
            ballots = self.convert_ballot_to_one_hot(ballots)
            clusters.append(ballots)
            
        return clusters
    
    def n_projects_in_cluster(self, cluster_size):
        # the lower cluster_independence is the more dependent the number of projects are in therms of clustersize. 
        return int(np.clip(random.normal(self.size_none_empty_projects * (cluster_size / self.max_voters_per_cluster), self.cluster_independence, 1), 1, self.size_none_empty_projects))
 
    def generate_project_set(self, n_projects_in_cluster):
        return self.rng.choice(self.n_projects, n_projects_in_cluster, p=self.votes_per_project_dist, replace=False, shuffle=False)
        
    def approvals_per_voter(self, n_projects_in_cluster, cluster_size):
        # spread_of_approvals determines the spread of approvals per voter within a cluster.
        return np.clip(random.normal(self.avg_approvals_per_voter + 1, self.spread_of_approvals, cluster_size), 1, n_projects_in_cluster).astype(int)

    def gen_cluster_project_dist(self, project_set):
        # distribution for all projects (small values for non-cluster-projects)
        new_dist = np.maximum(self.votes_per_project_dist[project_set] + random.normal(self.avg_diff_cluster_all_voters_project_dist, self.std_diff_cluster_all_voters_project_dist, len(project_set)), self.noise)
        return new_dist / sum(new_dist)
    
    def gen_cluster_project_dist_noise(self, cluster_project_dist, project_set):
        noise_projects = list(set(range(self.n_projects)) - set(project_set))
        cpdn = np.array(list(cluster_project_dist) + list(np.clip(random.normal(self.noise, self.noise/2 ,len(noise_projects)), 10**-8, 3*self.noise)))
        return cpdn / sum(cpdn), list(project_set) + noise_projects
    
    def gen_cluster_ballot(self, project_set, approvals_per_voter, cluster_project_dist):
        return [self.rng.choice(project_set, apv, p=cluster_project_dist, replace=False, shuffle=False) for apv in approvals_per_voter]
 
    def __call__(self):
        return self.make_clusters()

In [11]:
VOTERS_PER_CLUSTER = list(range(1500, 100, -100))
VOTES_PER_PROJECT = list(range(5000, 1500, -300))

tmp = data_generator(VOTERS_PER_CLUSTER, VOTES_PER_PROJECT)()
print(Counter([z for x in tmp for y in x for z in np.where(y==1)[0]]))

data = [y for x in tmp for y in  x]

ValueError: probabilities are not non-negative

In [ ]:
def generate_ballot(m, p=0.5):
    return (random.rand(m)+(p-0.5)).round().astype(int)

class Cluster():
    def __init__(self, m, n, permutation_rate=0.1, p=0.5):
        self.m = m
        m = int(m * 0.95)
        self.n = n
        self.permutation_rate = permutation_rate
        self._index_map = np.array(sample(range(self.m), m))
        self._mean = generate_ballot(m, p)
        self.__create_n_voter_ballots(m)

        self._mean, tmp = np.zeros(self.m), self._mean
        self._mean[self._index_map] = tmp
        
    def __create_n_voter_ballots(self, m):
        self._ballots = np.zeros((self.n, self.m))
        for i in range(self.n):
            self._ballots[i,self._index_map] = self.__permutate_ballot(m)
        
    def __permutate_ballot(self, m):
        # point permutate the ballot up to 10 times
        new_ballot = copy.copy(self._mean)
        for _ in range(random.randint(self.permutation_rate*m+1)):
            new_ballot[random.randint(m)] = 1 - new_ballot[random.randint(m)]
            
        return new_ballot
    
    @property
    def ballots(self):
        return self._ballots
    
    @property
    def cluster_ballot(self):
        return self._mean
    
    def __repr__(self):
        return str(self._ballots)
    
    def statistics(self):
        rcParams['figure.figsize'] = 7, 4
        rcParams['figure.dpi'] = 75
        rcParams.update({'font.size': 10})
        print(f"Probability that a voter votes the same for one project compared to the cluster ballot = {1-np.mean(np.abs(self._ballots-self._mean))}")
        x,y = np.unique(self._ballots, return_counts=True, axis=0)
        print(f"Number of unique ballets in this cluster = {len(x)}")
        print(f"Number of voters that vote the cluster ballot = {max(y)}")
        plt.plot(*np.unique(np.abs(self._ballots - self._mean).sum(1), return_counts=True), '-o')
        plt.xlabel("difference between voter ballot and cluster ballot")
        plt.ylabel("number of voters")
        plt.show()
        plt.plot(self._mean*100, 'o',label="cluster ballot")
        plt.plot(self._ballots.mean(0)*100,'x',label="voters")
        plt.legend()
        plt.xlim(0,self.m)
        plt.xlabel("Projects")
        plt.ylabel("Change of approving a project(%)")
        plt.show()
        
    